In [106]:
import sys
sys.path.append('../src')

# --- CEL 1: imports en data ---
import pandas as pd
import numpy as np
import yfinance as yf
import importlib
import indicators

importlib.reload(indicators)
from indicators import (
    sma, ema, rsi, macd, bollinger_bands, adx, atr, stochastic, cci,
    williams_r, roc, obv, mfi, cmf, supertrend, keltner_channels, donchian_channels
)

ticker = "HIVE"
df = yf.download(ticker, period="6mo", interval="1d", auto_adjust=False)

# Kolomnamen corrigeren als er een MultiIndex is
df.columns = df.columns.get_level_values(0)

# Zorg dat alle relevante kolommen bestaan
needed_cols = ["Open", "High", "Low", "Close", "Volume"]
for c in needed_cols:
    if c not in df.columns:
        raise KeyError(f"Kolom '{c}' ontbreekt in Yahoo Finance data")

# Flatten mogelijke 2D arrays
for c in needed_cols:
    df[c] = np.ravel(df[c].values)

# laat alle rijen staan, zodat we laatste rijen behouden
df = df.copy()

df.tail(3)


[*********************100%***********************]  1 of 1 completed

Extended indicators module loaded


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2025-10-15,6.81,6.81,7.08,6.34,7.03,28549500
2025-10-16,5.84,5.84,6.77,5.82,6.73,33339800
2025-10-17,5.71,5.71,5.82,4.97,4.97,37080200


In [107]:
# --- CEL 2: indicatoren berekenen ---
# Trend
df["EMA20"] = ema(df["Close"], 20)
df["EMA50"] = ema(df["Close"], 50)
adx_df = adx(df["High"], df["Low"], df["Close"], window=14)
df[["ADX", "Plus_DI", "Minus_DI"]] = adx_df[["ADX", "Plus_DI", "Minus_DI"]]
st_df = supertrend(df["High"], df["Low"], df["Close"], atr_period=10, multiplier=3)
df[["SuperTrend", "ST_Dir"]] = st_df[["SuperTrend", "Direction"]]

# Momentum
df["RSI14"] = rsi(df["Close"], 14)
macd_df = macd(df["Close"], 12, 26, 9)
df[["MACD", "MACD_Signal", "MACD_Hist"]] = macd_df[["MACD", "Signal", "Histogram"]]
df["ROC10"] = roc(df["Close"], 10)
sto = stochastic(df["High"], df["Low"], df["Close"], k_window=14, d_window=3)
df[["STO_K", "STO_D"]] = sto[["K", "D"]]
df["CCI20"] = cci(df["High"], df["Low"], df["Close"], window=20)
df["WR14"] = williams_r(df["High"], df["Low"], df["Close"], window=14)

# Volatiliteit en kanalen
df["ATR14"] = atr(df["High"], df["Low"], df["Close"], window=14)
bb = bollinger_bands(df["Close"], window=20, num_std=2)
df[["BB_Upper", "BB_Middle", "BB_Lower"]] = bb[["Upper", "Middle", "Lower"]]
kc = keltner_channels(df["High"], df["Low"], df["Close"], window=20, atr_period=10, multiplier=2)
df[["KC_Upper", "KC_Middle", "KC_Lower"]] = kc[["Upper", "Middle", "Lower"]]
don = donchian_channels(df["High"], df["Low"], window=20)
df[["Don_Upper", "Don_Middle", "Don_Lower"]] = don[["Upper", "Middle", "Lower"]]

# Volume / Money Flow
df["OBV"] = obv(df["Close"], df["Volume"])
df["MFI14"] = mfi(df["High"], df["Low"], df["Close"], df["Volume"], window=14)
df["CMF20"] = cmf(df["High"], df["Low"], df["Close"], df["Volume"], window=20)

df = df.copy()
df.tail(3)


Price,Adj Close,Close,High,Low,Open,Volume,EMA20,EMA50,ADX,Plus_DI,...,BB_Lower,KC_Upper,KC_Middle,KC_Lower,Don_Upper,Don_Middle,Don_Lower,OBV,MFI14,CMF20
Date,,,,,,,,,,,,,,,,,,,,,
2025-10-15,6.81,6.81,7.08,6.34,7.03,28549500,5.324969,4.094910,56.697987,53.541420,...,2.254283,6.778969,5.324969,3.870969,7.84,5.64,3.44,709706800.0,91.357445,0.298357
2025-10-16,5.84,5.84,6.77,5.82,6.73,33339800,5.374020,4.163345,58.086662,48.903514,...,2.383429,6.968020,5.374020,3.780020,7.84,5.64,3.44,676367000.0,88.875627,0.295245
2025-10-17,5.71,5.71,5.82,4.97,4.97,37080200,5.406018,4.223998,57.875975,43.879673,...,2.536370,7.106018,5.406018,3.706018,7.84,5.64,3.44,639286800.0,83.694557,0.332036


In [108]:
# --- CEL 3: hulpfuncties ---
import numpy as np

def slope(series, lookback=5):
    """Lineaire regressiehelling van de laatste n waarden."""
    if len(series) < lookback:
        return np.nan
    x = np.arange(lookback)
    y = series.tail(lookback).values
    denom = np.sum((x - x.mean())**2)
    if denom == 0:
        return 0.0
    m = np.sum((x - x.mean()) * (y - y.mean())) / denom
    return float(m)

def near(a, b, pct=0.02):
    """Controleer of a binnen ±pct van b ligt."""
    return abs(a - b) <= pct * b

def rising(series, n=3):
    """Controleer of de laatste n waarden stijgend zijn."""
    if len(series) < n:
        return False
    s = series.tail(n)
    return all(s.iloc[i] > s.iloc[i-1] for i in range(1, len(s)))



In [109]:
# --- CEL 4: scoresysteem ---
WEIGHTS = {
    "trend": 35,
    "momentum": 30,
    "volume": 20,
    "volatility": 15
}

def score_row(row, window_df):
    # window_df: laatste N rijen om "rising" en "slope" te beoordelen
    scores = {"trend":0.0, "momentum":0.0, "volume":0.0, "volatility":0.0}
    notes = []

    # TREND (max 35)
    if row["Close"] > row["EMA20"]:
        scores["trend"] += 10; notes.append("Close > EMA20")
    if row["EMA20"] > row["EMA50"]:
        scores["trend"] += 8; notes.append("EMA20 > EMA50")
    if row["Close"] > row["EMA50"]:
        scores["trend"] += 7; notes.append("Close > EMA50")
    if row["ADX"] > 20 and row["Plus_DI"] > row["Minus_DI"]:
        scores["trend"] += 5; notes.append("ADX>20 en +DI>-DI")
    if row["ST_Dir"] == 1:
        scores["trend"] += 5; notes.append("SuperTrend bullish")

    # MOMENTUM (max 30)
    if row["MACD"] > row["MACD_Signal"]:
        scores["momentum"] += 10; notes.append("MACD boven signaal")
    if rising(window_df["MACD_Hist"], n=3):
        scores["momentum"] += 5; notes.append("Histogram stijgt")
    if 45 <= row["RSI14"] <= 65:
        scores["momentum"] += 5; notes.append("RSI neutraal-sterk")
    # kruis omhoog rond 50
    rsi_last5 = window_df["RSI14"].tail(5)
    if (rsi_last5.iloc[-2] < 50) and (rsi_last5.iloc[-1] >= 50):
        scores["momentum"] += 5; notes.append("RSI kruis boven 50")
    if row["ROC10"] > 0:
        scores["momentum"] += 5; notes.append("ROC10 positief")

    # VOLUME/FLOW (max 20)
    if slope(window_df["OBV"], 5) > 0:
        scores["volume"] += 7; notes.append("OBV stijgend")
    if 40 <= row["MFI14"] <= 70:
        scores["volume"] += 5; notes.append("MFI gebalanceerd")
    if rising(window_df["MFI14"], 3):
        scores["volume"] += 3; notes.append("MFI stijgt")
    if row["CMF20"] > 0:
        scores["volume"] += 5; notes.append("CMF>0")

    # VOLATILITEIT/BREAKOUT (max 15)
    if row["Close"] > row["KC_Middle"] or row["Close"] > row["BB_Middle"]:
        scores["volatility"] += 5; notes.append("Boven midden kanaal")
    # Donchian breakout of bijna breakout
    if near(row["Close"], row["Don_Upper"], pct=0.0) or row["Close"] >= row["Don_Upper"]:
        scores["volatility"] += 7; notes.append("Donchian breakout")
    elif near(row["Close"], row["Don_Upper"], pct=0.02):
        scores["volatility"] += 3; notes.append("Dicht bij 20d high")
    # Rustige volatiliteit tijdens uptrend
    atr_slope = slope(window_df["ATR14"], 5)
    if atr_slope <= 0 and row["EMA20"] > row["EMA50"]:
        scores["volatility"] += 3; notes.append("ATR daalt in uptrend")

    # Totaal
    total = sum(scores[k] for k in scores.keys())
    # normaliseren naar 0–100 exact op basis van ingestelde maxima
    max_total = sum(WEIGHTS.values())  # 100
    score_pct = round(100 * total / max_total, 1)

    if score_pct >= 70:
        signal = "BUY"
    elif 55 <= score_pct < 70:
        signal = "WATCH / BUY ON STRENGTH"
    elif 40 <= score_pct < 55:
        signal = "HOLD / NEUTRAL"
    else:
        signal = "AVOID / TAKE-PROFIT"

    return score_pct, signal, notes

def compute_signal(df, lookback=5):
    row = df.iloc[-1]
    window_df = df.tail(max(lookback, 5))
    score, signal, notes = score_row(row, window_df)
    out = {
        "Price": round(row["Close"], 2),
        "Score": score,
        "Signal": signal,
        "RSI": round(row["RSI14"], 1),
        "MACD": round(row["MACD"], 2),
        "MACD_Signal": round(row["MACD_Signal"], 2),
        "ADX": round(row["ADX"], 1),
        "MFI": round(row["MFI14"], 1),
        "CMF": round(row["CMF20"], 3),
        "OBV_slope_5": round(slope(window_df["OBV"], 5), 2)
    }
    return out, notes


In [110]:
# --- CEL 5: draai de score ---
# verwijder alleen rijen waar de belangrijkste waarden ontbreken
core_cols = ["Close","EMA20","EMA50","RSI14","MACD","ADX","MFI14"]
df = df.dropna(subset=core_cols)

if len(df) == 0:
    raise ValueError("Geen geldige rijen over; controleer of indicatorberekening gelukt is.")

summary, reasons = compute_signal(df, lookback=5)


In [111]:
# --- CEL 6: regels voor instap/exit ---
def entry_exit_guidance(df, score_threshold=70):
    row = df.iloc[-1]
    cond_buy = (
        row["Close"] > row["EMA20"] and
        row["EMA20"] > row["EMA50"] and
        row["MACD"] > row["MACD_Signal"] and
        row["ADX"] > 20 and
        row["ST_Dir"] == 1
    )
    entry = "Koop op doorbraak boven recent high" if cond_buy else "Wachten op bevestiging (Close > EMA20 en MACD>Signaal)"

    # stop en target op basis van volatiliteit
    atr = row["ATR14"]
    stop = round(row["Close"] - 1.5 * atr, 2)
    target1 = round(row["Close"] + 1.5 * atr, 2)
    target2 = round(row["Close"] + 3.0 * atr, 2)

    return {
        "Entry_rule": entry,
        "Stop_loss": stop,
        "Target1": target1,
        "Target2": target2
    }

guidance = entry_exit_guidance(df)
print(guidance)


{'Entry_rule': 'Koop op doorbraak boven recent high', 'Stop_loss': np.float64(4.68), 'Target1': np.float64(6.74), 'Target2': np.float64(7.78)}


In [112]:
# --- CEL 5: draai de score ---
core_cols = ["Close","EMA20","EMA50","RSI14","MACD","ADX","MFI14"]
df = df.dropna(subset=core_cols).copy()

print("Aantal rijen in df:", len(df))
if len(df) == 0:
    raise ValueError("Geen geldige rijen over; controleer of indicatorberekening gelukt is.")

summary, reasons = compute_signal(df, lookback=5)

print("\n=== SWING TRADING SCORE ===")
print(f"Ticker: {ticker}\n")
for k, v in summary.items():
    print(f"{k}: {v}")

print("\n=== REDENEN ===")
if len(reasons) == 0:
    print("Geen actieve signalen gevonden (neutraal).")
else:
    for r in reasons:
        print(f"- {r}")


Aantal rijen in df: 113

=== SWING TRADING SCORE ===
Ticker: HIVE

Price: 5.71
Score: 60.0
Signal: WATCH / BUY ON STRENGTH
RSI: 68.7
MACD: 0.81
MACD_Signal: 0.81
ADX: 57.9
MFI: 83.7
CMF: 0.332
OBV_slope_5: -20204470.0

=== REDENEN ===
- Close > EMA20
- EMA20 > EMA50
- Close > EMA50
- ADX>20 en +DI>-DI
- SuperTrend bullish
- MACD boven signaal
- ROC10 positief
- CMF>0
- Boven midden kanaal
